In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
def ensemble_models(models,model_input):
    outputs = [model(model_input) for model in models]
    
    avrg = average(outputs)
    
    model = Model(inputs=model_input, outputs=avrg, name='ensemble_model')
    
    return model

In [6]:
vgg16_model_GAP = load_model('vgg16_GAP_final.h5')
vgg16_model_GAP.name = 'vgg16_model_GAP'

for layer in vgg16_model_GAP.layers:
    layer.name = layer.name + '_GAP_16'
    print(layer.name)

vgg19_model_GAP = load_model('vgg19_GAP_final.h5')
vgg19_model_GAP.name = 'vgg19_model_GAP'

for layer in vgg19_model_GAP.layers:
    layer.name = layer.name + '_GAP_19'
    print(layer.name)

vgg16_model_FC = load_model('vgg16_model_0427_openall.h5')
vgg16_model_FC.name = 'vgg16_model_FC'
for layer in vgg16_model_FC.layers:
    layer.name = layer.name + '_FC_16'
    print(layer.name)

vgg19_model_FC = load_model('vgg19_final_model.h5')
vgg19_model_FC.name = 'vgg19_model_FC'
for layer in vgg19_model_FC.layers:
    layer.name = layer.name + '_FC_19'
    print(layer.name)

# models = [vgg16_model_GAP,vgg19_model_GAP]
models = [vgg16_model_GAP,vgg19_model_GAP,vgg16_model_FC,vgg19_model_FC]

model_input = Input(shape=(224,224,3))

ensemble_model = ensemble_models(models,model_input)

ensemble_model.summary()


input_1_GAP_16
block1_conv1_GAP_16
block1_conv2_GAP_16
block1_pool_GAP_16
block2_conv1_GAP_16
block2_conv2_GAP_16
block2_pool_GAP_16
block3_conv1_GAP_16
block3_conv2_GAP_16
block3_conv3_GAP_16
block3_pool_GAP_16
block4_conv1_GAP_16
block4_conv2_GAP_16
block4_conv3_GAP_16
block4_pool_GAP_16
block5_conv1_GAP_16
block5_conv2_GAP_16
block5_conv3_GAP_16
block5_pool_GAP_16
global_average_pooling2d_1_GAP_16
dense_1_GAP_16
input_1_GAP_19
block1_conv1_GAP_19
block1_conv2_GAP_19
block1_pool_GAP_19
block2_conv1_GAP_19
block2_conv2_GAP_19
block2_pool_GAP_19
block3_conv1_GAP_19
block3_conv2_GAP_19
block3_conv3_GAP_19
block3_conv4_GAP_19
block3_pool_GAP_19
block4_conv1_GAP_19
block4_conv2_GAP_19
block4_conv3_GAP_19
block4_conv4_GAP_19
block4_pool_GAP_19
block5_conv1_GAP_19
block5_conv2_GAP_19
block5_conv3_GAP_19
block5_conv4_GAP_19
block5_pool_GAP_19
global_average_pooling2d_1_GAP_19
dense_1_GAP_19
input_1_FC_16
block1_conv1_FC_16
block1_conv2_FC_16
block1_pool_FC_16
block2_conv1_FC_16
block2_conv2_

In [7]:
test_model(ensemble_model,test_path=test_path,csv='vgg16_and_vgg19_all_submission_0506.csv')

In [10]:
# model1 = load_model('simple_cnn_final_model.h5')
# model2= load_model('simple_cnn_final_model.h5')

# input1 = model1.input
# input2 = model2.input

# # layer1 = model1.get_layer(index=-2)
# # layer2 = model2.get_layer(index=-2)

# # avrg_layer = average([layer1,layer2])
# # out = Dense(10, activation='softmax')(avrg_layer)
# output1 = model1.output
# output2 = model2.output

# out = Average()([output1,output2])

# model = Model(inputs=[input1,input2],outputs=out)
# model1.summary()

In [11]:
# def get_features(model):
#     image_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

#     train_generator = image_datagen.flow_from_directory(
#                                         train_path,
#                                         target_size=image_size,
#                                         batch_size=batch_size,
#                                         class_mode='categorical',
#                                         shuffle=False)

#     validation_generator = image_datagen.flow_from_directory(
#                                         validation_path,
#                                         target_size=image_size,
#                                         batch_size=batch_size,                                    
#                                         class_mode='categorical',
#                                         shuffle=False)
    
#     train_data = model.predict_generator(train_generator,nb_train_samples)
#     train_label = train_generator.classes
    
#     validation_data = model.predict_generator(validation_generator,nb_val_samples)
#     validation_label = validation_generator.classes
    
#     return train_data,train_label,validation_data,validation_label

In [12]:
# vgg16_model = load_model('vgg16_final_model.h5')
# vgg16_model.name = 'vgg16_model'

# vgg19_model = load_model('vgg19_final_model.h5')
# vgg19_model.name = 'vgg19_model'

# train_data = []
# train_label = []
# validation_data = []
# validation_label = []

# for model in [vgg16_model,vgg19_model]:
#     td,tl,vd,vl = get_features(model)
#     train_data.append(td)
#     train_label.append(tl)
#     validation_data.append(vd)
#     validation_label.append(vl)
    
# train_data = np.concatenate(train_data, axis=1)
# train_label = np.concatenate(train_label, axis=1)
# validation_data = np.concatenate(validation_data, axis=1)
# validation_label = np.concatenate(validation_label, axis=1)

# (train_data,train_label) = shuffle(train_data,train_label)

# model_input = Input(shape=train_data.shape[1:])
# x = Dropout(0.5)(model_input)
# x = Dense(10, activation='softmax')(x)
# merge_model = Model(inputs=model_input,outputs=x)

# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(train_data,train_label,batch_size=batch_size,epochs=10,
#           validation_data=(validation_data,validation_label),
#           shuffle=True)

In [13]:
# print(train_data.shape)
# print(validation_data.size())

In [14]:
# test_model(ensenble_model,test_path=test_path,csv='vgg16_and_vgg19_submission.csv')